In [15]:
from utilities import *
from model import *
from metrics import Metrics
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
from tqdm import tqdm
import time
import multiprocessing
from sklearn import svm
from gensim.test.utils import get_tmpfile
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.sparse import csr_matrix as sparse
from os import listdir
from os.path import isfile, join


In [11]:
doc2vec_model.model.window

2

In [2]:
file_dir = "/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/data"
data = DataHandler(file_dir)

filename = "model_dm=0,hs=1,min_count=10,vector_size=100,window=2.bin"
loc = '/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/'
doc2vec_model = MyDoc2Vec()
doc2vec_model.load(loc+filename)

file_dir_our      = "/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/data"
our_data      = DataHandler(file_dir_our)
our_data.readOurData()

train, test   = our_data.blind_test()

cv_train      = DataHandler()
cv_train(train.x_train, train.y_train)

blind_test = DataHandler()
blind_test(test.x_train, test.y_train)

vector_data = DataHandler()
vector_data(DataHandler.applyDoc2Vec(cv_train.x_data, doc2vec_model.model), cv_train.y_data)

vector_data_test = DataHandler()
vector_data_test(DataHandler.applyDoc2Vec(blind_test.x_data, doc2vec_model.model), blind_test.y_data)


100%|██████████| 2000/2000 [00:01<00:00, 1999.91it/s]


In [3]:
my_svm = MySVM(gamma=0.01, kernel = 'linear')
my_svm.train(vector_data.x_data, vector_data.y_data)

vector_data.roundRobinSplit(10)
cv_acc      = Metrics.roundRobinCV(vector_data, 10, MySVM, gamma=0.01, kernel = 'rbf')
predictions = my_svm.predict(DataHandler.applyDoc2Vec(test.x_train, doc2vec_model.model))


100%|██████████| 10/10 [00:03<00:00,  2.73it/s]


In [4]:
Metrics.getAccuracy(predictions, vector_data_test.y_data)

0.795

In [5]:
np.mean(np.array(cv_acc))

0.8783333333333333

In [6]:
cv_acc

[0.8611111111111112,
 0.8944444444444445,
 0.8777777777777778,
 0.8444444444444444,
 0.8777777777777778,
 0.8555555555555555,
 0.9,
 0.9,
 0.9222222222222223,
 0.85]

# Figure out what the actual cross validated performance is

In [13]:
model_path = '/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/'
onlyfiles = [f for f in listdir(model_path) if isfile(join(model_path, f))]

In [19]:
model_path = '/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/'
onlyfiles = [f for f in listdir(model_path) if isfile(join(model_path, f))]

for filename in tqdm(onlyfiles):
    print(model_path+filename)

100%|██████████| 36/36 [00:00<00:00, 8838.90it/s]

/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/model_dm=0,hs=0,min_count=1,vector_size=100,window=1.bin
/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/model_dm=0,hs=0,min_count=1,vector_size=100,window=2.bin
/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/model_dm=0,hs=0,min_count=1,vector_size=100,window=5.bin
/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/model_dm=0,hs=0,min_count=1,vector_size=50,window=1.bin
/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/model_dm=0,hs=0,min_count=1,vector_size=50,window=2.bin
/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/model_dm=0,hs=0,min_count=1,vector_size=50,window=5.bin
/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/model_dm=0,hs=0,min_count=10,vector_size=100,window=1.bin
/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursewor

In [29]:
file_dir = "/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/data"
data = DataHandler(file_dir)
our_data      = DataHandler(file_dir_our)
our_data.readOurData()

train, test   = our_data.blind_test()

cv_train      = DataHandler()
cv_train(train.x_train, train.y_train)

blind_test = DataHandler()
blind_test(test.x_train, test.y_train)

model_path = '/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/'
onlyfiles = [f for f in listdir(model_path) if isfile(join(model_path, f))]
onlyfiles = [file for file in onlyfiles if re.search(".bin$",file)]
results = {}

my_svm = MySVM(gamma=0.01, kernel = 'linear')


for filename in tqdm(onlyfiles):
    doc2vec_model = MyDoc2Vec()
    doc2vec_model.load(model_path+filename)

    vector_data = DataHandler()
    vector_data(DataHandler.applyDoc2Vec(cv_train.x_data, doc2vec_model.model), cv_train.y_data)

    vector_data_test = DataHandler()
    vector_data_test(DataHandler.applyDoc2Vec(blind_test.x_data, doc2vec_model.model), blind_test.y_data)
    
    my_svm.train(vector_data.x_data, vector_data.y_data)

    vector_data.roundRobinSplit(10)
    cv_acc      = Metrics.roundRobinCV(vector_data, 10, MySVM, gamma=0.01, kernel = 'linear')
    predictions = my_svm.predict(vector_data_test.x_data)
    
    results[filename] = [Metrics.getAccuracy(predictions, vector_data_test.y_data), np.mean(np.array(cv_acc))]


100%|██████████| 2000/2000 [00:01<00:00, 1626.22it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:03,  2.53it/s]

 20%|██        | 2/10 [00:00<00:03,  2.53it/s]

 30%|███       | 3/10 [00:01<00:02,  2.64it/s]

 40%|████      | 4/10 [00:01<00:02,  2.80it/s]

 50%|█████     | 5/10 [00:01<00:01,  2.88it/s]

 60%|██████    | 6/10 [00:02<00:01,  2.88it/s]

 70%|███████   | 7/10 [00:02<00:01,  2.89it/s]

 80%|████████  | 8/10 [00:02<00:00,  2.92it/s]

 90%|█████████ | 9/10 [00:03<00:00,  2.63it/s]

100%|██████████| 10/10 [00:03<00:00,  2.71it/s]

  3%|▎         | 1/36 [01:38<57:44, 98.99s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:03,  2.70it/s]

 20%|██        | 2/10 [00:00<00:02,  2.75it/s]

 30%|███       | 3/10 [00:01<00:02,  2.75it/s]

 40%|████      | 4/10 [00:01<00:02,  2.74it/s]

 50%|█████     | 5/10 [00:01<00:01,  2.62it/s]

 60%|██████    | 6/10 [00:02<00:01,  2.53it/s]

 70%|██

 80%|████████  | 8/10 [00:01<00:00,  6.64it/s]

 90%|█████████ | 9/10 [00:01<00:00,  6.43it/s]

100%|██████████| 10/10 [00:01<00:00,  6.67it/s]

 33%|███▎      | 12/36 [16:32<29:56, 74.85s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.07it/s]

 20%|██        | 2/10 [00:00<00:02,  3.13it/s]

 30%|███       | 3/10 [00:00<00:02,  3.00it/s]

 40%|████      | 4/10 [00:01<00:01,  3.04it/s]

 50%|█████     | 5/10 [00:01<00:01,  3.08it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.22it/s]

 70%|███████   | 7/10 [00:02<00:00,  3.23it/s]

 80%|████████  | 8/10 [00:02<00:00,  3.29it/s]

 90%|█████████ | 9/10 [00:02<00:00,  3.30it/s]

100%|██████████| 10/10 [00:03<00:00,  3.16it/s]

 36%|███▌      | 13/36 [17:53<29:21, 76.59s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.06it/s]

 20%|██        | 2/10 [00:00<00:02,  2.96it/s]

 30%|███       | 3/10 [00:01<00:02,  3.03it/s]

 40%|████      | 4/10 [00:01<00:01,  3.09it/s]

 50%

 50%|█████     | 5/10 [00:01<00:01,  3.57it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.67it/s]

 70%|███████   | 7/10 [00:01<00:00,  3.78it/s]

 80%|████████  | 8/10 [00:02<00:00,  3.82it/s]

 90%|█████████ | 9/10 [00:02<00:00,  3.78it/s]

100%|██████████| 10/10 [00:02<00:00,  3.73it/s]

 69%|██████▉   | 25/36 [46:50<32:13, 175.73s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.24it/s]

 20%|██        | 2/10 [00:00<00:02,  3.00it/s]

 30%|███       | 3/10 [00:00<00:02,  3.17it/s]

 40%|████      | 4/10 [00:01<00:01,  3.18it/s]

 50%|█████     | 5/10 [00:01<00:01,  3.28it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.31it/s]

 70%|███████   | 7/10 [00:02<00:00,  3.43it/s]

 80%|████████  | 8/10 [00:02<00:00,  3.40it/s]

 90%|█████████ | 9/10 [00:02<00:00,  3.23it/s]

100%|██████████| 10/10 [00:03<00:00,  3.29it/s]

 72%|███████▏  | 26/36 [49:22<28:06, 168.64s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.28it/s]

 2

In [27]:
type(re.search(".bin$",'model_dm=0,hs=0,min_count=1,vector_size=100,window=1.bin.'))

NoneType

In [30]:
results

{'model_dm=0,hs=0,min_count=1,vector_size=100,window=1.bin': [0.785,
  0.8377777777777778],
 'model_dm=0,hs=0,min_count=1,vector_size=100,window=2.bin': [0.795,
  0.8377777777777776],
 'model_dm=0,hs=0,min_count=1,vector_size=100,window=5.bin': [0.795,
  0.8455555555555556],
 'model_dm=0,hs=0,min_count=1,vector_size=50,window=1.bin': [0.775,
  0.8444444444444444],
 'model_dm=0,hs=0,min_count=1,vector_size=50,window=2.bin': [0.795,
  0.8516666666666668],
 'model_dm=0,hs=0,min_count=1,vector_size=50,window=5.bin': [0.795, 0.85],
 'model_dm=0,hs=0,min_count=10,vector_size=100,window=1.bin': [0.855,
  0.8655555555555556],
 'model_dm=0,hs=0,min_count=10,vector_size=100,window=2.bin': [0.81,
  0.8738888888888889],
 'model_dm=0,hs=0,min_count=10,vector_size=100,window=5.bin': [0.8,
  0.8733333333333334],
 'model_dm=0,hs=0,min_count=10,vector_size=50,window=1.bin': [0.83, 0.875],
 'model_dm=0,hs=0,min_count=10,vector_size=50,window=2.bin': [0.81,
  0.8761111111111111],
 'model_dm=0,hs=0,min_co

# Prepare vectors for online visualisation


In [31]:
file_dir = "/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/data"
data = DataHandler(file_dir)
our_data      = DataHandler(file_dir)
our_data.readOurData()

train, test   = our_data.blind_test()

cv_train      = DataHandler()
cv_train(train.x_train, train.y_train)

blind_test = DataHandler()
blind_test(test.x_train, test.y_train)

model_path = '/Users/igoradamski/Documents/cambridge/MLMI/nlp/coursework/nlp/doc2vec_models/'

model_name = 'model_dm=0,hs=1,min_count=10,vector_size=50,window=5.bin'

doc2vec_model = MyDoc2Vec()
doc2vec_model.load(model_path+model_name)

vector_data = DataHandler()
vector_data(DataHandler.applyDoc2Vec(cv_train.x_data, doc2vec_model.model), cv_train.y_data)

vector_data_test = DataHandler()
vector_data_test(DataHandler.applyDoc2Vec(blind_test.x_data, doc2vec_model.model), blind_test.y_data)

vector_data.roundRobinSplit(10)


100%|██████████| 2000/2000 [00:01<00:00, 1701.28it/s]


In [39]:
np.savetxt('data_vectors.tsv', vector_data.x_data, delimiter='\t')
np.savetxt('data_vectors_labels.tsv', np.array([[idx, val] for idx,val in enumerate(vector_data.y_data)]), delimiter='\t', header="id,label")

TypeError: an integer is required (got type list)